Continuar a partir de 7:40 (aula 10)

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from statsmodels.tsa.ar_model import AutoReg as AR

In [50]:
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

### Exercício 

A sua missão é prever 3 meses de uma série temporal de 2 anos, com início no dia 2020-01-01 e término no dia 2021-12-30, usando os mesmos parâmetros de quebra de premissas apresentadas nas aulas anteriores

### Entregáveis:

1. Código python para criar a série temporal do exercício

In [ ]:
# Configurações da serie
np.random.seed(42)
n = 500

# criacao da series perfeita
trend = np.linspace( 0, 0, n)
noise = np.random.normal( 0, 1, n) 
serie_perfeita = trend + noise
dates = pd.date_range( start='2020-01-01', end='2021-12-20', periods=n )
serie_perfeita = pd.Series( serie_perfeita, index=dates, name='serie_perfeita')

# Quebra Premissa 1: Linearidade 
trend_break = np.linspace( 0, 10, n )
serie_nao_linear = serie_perfeita + trend_break

# Quebra Premissa 2: Estacionariedade 
seasonality = 3 * np.sin( 2 * np.pi * np.arange( n ) / 50 )
serie_nao_estacionaria = serie_perfeita + seasonality

# Quebra Premissa 3: Autocorrelacao dos residucos 
autoregressive = np.zeros( n)
autoregressive[0] = noise[0]
for t in range( 1, n ):
 autoregressive[t] = 0.8 * autoregressive[t-1] + np.random.normal( 0, 0.5)
serie_nao_autocorrelacao = serie_perfeita + autoregressive

# Quebra Premissa 4: Homoscedasticidade 
non_normal = noise * np.linspace( 1, 3, n )
serie_nao_homoscedastica = serie_perfeita + non_normal

# Quebra Premissa 5: Não nomralidade dos residuos 
non_normal_noise = np.random.exponential( scale=1, size= n )
serie_nao_normal = trend + non_normal_noise

# Combinar as series
serie_final = (
 serie_perfeita
 + trend_break
 + seasonality
 + autoregressive
 + non_normal
 + non_normal_noise
)
serie_final = pd.Series( serie_final, index=dates, name='serie_final' )
serie_final = serie_final - serie_final.min() + 1

# Visualizacao das series
plt.figure( figsize=( 16, 8 ) )
sns.lineplot( serie_final )

In [51]:
# === 1. CRIAÇÃO DA SÉRIE COM QUEBRAS DE PREMISSAS ===
np.random.seed(42)
n = 500
trend = np.linspace(0, 0, n)
noise = np.random.normal(0, 1, n)
serie_perfeita = trend + noise
dates = pd.date_range(start='2020-01-01', end='2021-12-30', periods=n)
serie_perfeita = pd.Series(serie_perfeita, index=dates, name='serie_perfeita')

trend_break = np.linspace(0, 10, n)
seasonality = 3 * np.sin(2 * np.pi * np.arange(n) / 50)
autoregressive = np.zeros(n)
autoregressive[0] = noise[0]
for t in range(1, n):
    autoregressive[t] = 0.8 * autoregressive[t - 1] + np.random.normal(0, 0.5)
non_normal = noise * np.linspace(1, 3, n)
non_normal_noise = np.random.exponential(scale=1, size=n)
serie_final = (serie_perfeita + trend_break + seasonality + autoregressive + non_normal + non_normal_noise)
serie_final = pd.Series(serie_final, index=dates, name='serie_final')
serie_final = serie_final - serie_final.min() + 1

2. Código de separação dos conjuntos de treino, validação e teste

In [ ]:
# Divisão da Série em Conjuntos de Treinamento, Validação e Teste
train_size = int( 0.8 * len( serie_final ) )
validation_size = int( 0.1 * len( serie_final ) )

# Train-Val-Test Split
train = serie_final[:train_size]
validation = serie_final[train_size:train_size + validation_size]
test = serie_final[train_size + validation_size:]

In [ ]:
plt.figure( figsize=( 16, 8 ) )
sns.lineplot( train, label='Train', color='blue' )
sns.lineplot( validation, label='Validation', color='orange' )
sns.lineplot( test, label='Test', color='green' )
plt.title( 'Divisão da Série em Conjuntos de Treinamento, Validação e Teste' )
plt.xlabel( 'Data' )
plt.ylabel( 'Valores' )
plt.legend()

3. Código em python que realiza a previsão unicamente do próximo ponto da série de validação

In [ ]:
warnings.filterwarnings("ignore")
train_series = train.copy()

In [ ]:
# Previsão unicamente do próximo ponto da série de validação
predictions = []
actuals = []
for i in range(len(validation)):
    model = AR(train_series, lags=1)
    model_fit = model.fit()
    prediction = model_fit.forecast(steps=1).iloc[0]
    predictions.append(prediction)  # Append the prediction to the list
    actuals.append(validation.iloc[i])
    train_series = pd.concat([train_series, pd.Series([validation.iloc[i]], index=[validation.index[i]])])

# Avaliação do modelo
df = pd.DataFrame({'Predictions': predictions, 'Actuals': actuals}, index=validation.index)

In [ ]:
df

4. Código em python para realizar a previsão de todos os pontos da série de validação usando a técnica de Rolling Forecast Origin

In [ ]:
# Previsão de todos os pontos da série de validação usando a tecnica de rolling forecast
rolling_predictions = []
train_series = train.copy()
for i in range(len(validation)):
    model = AR(train_series, lags=1)
    model_fit = model.fit()
    prediction = model_fit.forecast(steps=1).iloc[0]
    rolling_predictions.append(prediction)  # Append the prediction to the list
    train_series = pd.concat([train_series, pd.Series([prediction], index=[validation.index[i]])])

# Avaliação do modelo
df_rolling = pd.DataFrame({'Rolling Predictions': rolling_predictions, 'Actuals': actuals}, index=validation.index)

In [ ]:
df_rolling

5. Código em python com a implementação das seguintes métricas de desempenho sobre os dados de validação: MAE, MAPE, RMSE, MSE, AIC e BIC

In [ ]:
# Implementando metricas de desempenho nos dados de validação
errors =  df['Actuals'] - df['Predictions']

rmse = np.sqrt((errors**2).mean())
mae = np.abs(errors).mean()
mape = (np.abs(errors) / df['Actuals']).mean() * 100
mse = (errors**2).mean()

aic = 2 * (len(df) * np.log(np.std(errors)) + 1)
bic = 2 * (len(df) * np.log(np.std(errors)) + 1 + 2 * 1)

In [ ]:
errors =  df['Actuals'] - df['Predictions']

rmse = np.sqrt((errors**2).mean())
mae = (np.abs(errors)).mean()
mape = np.mean(np.abs(errors / df['Actuals']))
log_likelihood = model_fit.llf
num_params = model_fit.params.shape[0]
num_obs = len(train_series)
aic = -2 * log_likelihood + 2 * num_params
bic = -2 * log_likelihood + num_params * np.log(num_obs)

In [ ]:
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")
print(f"MSE: {mse}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

In [ ]:
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")
print(f"MSE: {mse}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

6. Código em python com a implementação do FINE-TUNING usando a técnica de Random Forest para valores lags entre 1 e 5. Use 3 iterações.

In [ ]:
# Fine-tuning do ARIMA
lags_values = [1, 2, 3, 4, 5]
num_interations = 3
results_val = pd.DataFrame()

In [ ]:
for _ in range (num_interations):
    # Escolha de um valor aleatório para o lag
    lag = np.random.choice(lags_values)
    print(f"Testando parâmetro: {lag}")

    # Start Rolling Forecast Origin
    predictions = []
    actuals = []
    train_series = train.copy()
    for t in range(len(validation)):
        # Adicionando o valor real ao conjunto de treinamento
        # train_series = train_series.append(validation.iloc[t])
        
        # Definindo o modelo
        model = AR(train_series, lags=lag)
        
        # Treinamento do modelo
        model_fit = model.fit()
        
        # Previsão para o próximo passo
        forecast = model_fit.forecast(steps=1).iloc[0]
        # Armazenando a previsão e o valor real
        predictions.append(forecast)
        actuals.append(validation.iloc[t])

        # Adicionando o valor real ao conjunto de treinamento
        train_series = pd.concat([train_series, pd.Series(validation.iloc[t], index=[validation.index[t]])])

    # DataFrame para armazenar previsões e valores reais
    df_val = pd.DataFrame({
        'Previsão': predictions,
        'Real': actuals
    }, index=validation.index)

    # Computando o desempenho
    errors = df['Actuals'] - df['Predictions']
    rmse_val = np.sqrt((errors**2).mean())
    mae_val = (np.abs(errors)).mean()
    mape_val = np.mean(np.abs(errors / df['Actuals']))

    # Calculando AIC e BIC
    log_likelihood = model_fit.llf
    num_params = model_fit.params.shape[0]
    num_obs = len(train_series)

    aic_val = -2 * log_likelihood + 2 * num_params
    bic_val = -2 * log_likelihood + num_params * np.log(num_obs)

    performance = pd.DataFrame({'Lag': lag,
                                'RMSE_VAL': rmse_val, 
                                'MAE_VAL': mae_val, 
                                'MAPE_VAL': mape_val, 
                                'AIC_VAL': aic_val, 
                                'BIC_VAL': bic_val}, index=[0]
                                )
    results_val = pd.concat([results_val, performance], ignore_index=True)

# Exibindo os resultados
results_val.sort_values(by='RMSE_VAL', ascending=True).head(10)

7. Código em python que faça a união das séries de treinamento com a série de validação e faça o treinamento usando o melhor parâmetro definido no Fine-Turing.

In [ ]:
# Best parameter for fine-tuning
lag = 5

# Start Rolling Forecast Origin
predictions = []
actuals = []
results_test = pd.DataFrame()
train_series = pd.concat([train, validation])

# Rolling Forecast Origin
for t in range(len(test)):
    # Adicionando o valor real ao conjunto de treinamento
    # train_series = train_series.append(validation.iloc[t])

    # Definindo o modelo
    model = AR(train_series, lags=lag)

    # Treinamento do modelo
    model_fit = model.fit()

    # Previsão para o próximo passo
    forecast = model_fit.forecast(steps=1).iloc[0]
    # Armazenando a previsão e o valor real
    predictions.append(forecast)
    actuals.append(test.iloc[t])

    #Update the training set with the actual value
    train_series = pd.concat([train_series, pd.Series(test.iloc[t], index=[test.index[t]])])
# Adicionando o valor real ao conjunto de treinamento
# train_series = train_series.append(validation.iloc[t])

# DataFrame para armazenar previsões e valores reais
df_test = pd.DataFrame({'Previsão': predictions,'Real': actuals}, index=test.index)

# Computando o desempenho
errors = df['Actuals'] - df['Predictions']

rmse_test = np.sqrt((errors**2).mean())
mae_test = (np.abs(errors)).mean()
mape_test = np.mean(np.abs(errors / df['Actuals']))

# Calculando AIC e BIC
log_likelihood = model_fit.llf
num_params = model_fit.params.shape[0]
num_obs = len(train_series)

aic_test = -2 * log_likelihood + 2 * num_params
bic_test = -2 * log_likelihood + num_params * np.log(num_obs)

results_test = pd.DataFrame({'Lag': lag,
                             'RMSE_TEST': rmse_test,
                             'MAE_TEST': mae_test,
                             'MAPE_TEST': mape_test,
                             'AIC_TEST': aic_test,
                             'BIC_TEST': bic_test}, index=[0]
)

# Exibindo os resultados
results_test

8. Código em python da previsão sobre os dados de teste, bem como a tabela final com todas as métricas.

9. Modularizar o código em python do item 8, criando funções para simplificar e modularizar o código.